# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119225e+02     1.575026e+00
 * time: 0.6325180530548096
     1     1.097656e+01     9.764831e-01
 * time: 2.6560049057006836
     2    -1.172831e+01     1.083093e+00
 * time: 3.1477460861206055
     3    -3.359957e+01     7.974475e-01
 * time: 3.8971269130706787
     4    -4.710698e+01     6.365345e-01
 * time: 4.606719970703125
     5    -5.665303e+01     2.314068e-01
 * time: 5.329329013824463
     6    -5.966938e+01     1.999828e-01
 * time: 5.826551914215088
     7    -6.090017e+01     6.212948e-02
 * time: 6.305480003356934
     8    -6.137079e+01     5.870031e-02
 * time: 6.784229040145874
     9    -6.161454e+01     4.513103e-02
 * time: 7.319715976715088
    10    -6.180966e+01     2.891883e-02
 * time: 7.871808052062988
    11    -6.195768e+01     2.680782e-02
 * time: 8.411436080932617
    12    -6.205493e+01     1.961752e-02
 * time: 8.950335025787354
    13    -6.210369e+01     1.757180e-02
 * time: 9.489569902420044
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671059
    AtomicLocal         -18.8557675
    AtomicNonlocal      14.8522656
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485362 
    Xc                  -19.3336816

    total               -62.261666459151
